In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

## Collect NASDAQ and SEC Symbols

In [ ]:
all_companies = obb.equity.search("", provider="sec")
all_companies2 = obb.equity.search("", provider="nasdaq")
sec_symbols = all_companies.to_polars().select(["symbol", "name"])
nasdaq_symbols = all_companies2.to_polars().select(["symbol", "name"])

#### Combine SEC Symbols and NASDAQ Symbols

In [ ]:
# Assuming sec_symbols and nasdaq_symbols are Polars DataFrames with 'symbol' and 'name' columns
all_symbols = sec_symbols.join(
    nasdaq_symbols, on="symbol", how="full", coalesce=True
).with_columns([
    pl.coalesce("name", "name_right").alias("name"),
]).drop(["name_right"]).filter(pl.col("symbol").is_not_null())

# Display the first few rows of the combined DataFrame
all_symbols

#### Find Symbols in SEC that are NOT in NASDAQ

In [ ]:
# Assuming sec_symbols and nasdaq_symbols are Polars DataFrames with 'symbol' and 'name' columns
# Find the symbols in sec_symbols that are not in nasdaq_symbols
diff_symbols = (
    sec_symbols
    .join(nasdaq_symbols, on="symbol", how="anti")
    .select(["symbol", "name"])
    .unique(subset="symbol")
)

# Print the number of non-overlapping symbols
print(f"Number of non-overlapping symbols: {diff_symbols.height}")

# Display the resulting DataFrame
diff_symbols

## Collect `yfinance` & CBOE Indicies

In [ ]:
cboe_indices = obb.index.available(provider="cboe").to_polars()
yfinance_indices = obb.index.available(provider="yfinance").to_polars()

all_indices = cboe_indices.join(yfinance_indices, on="symbol", how="anti").select(["symbol", "name"])
all_indices

